In [2]:
from nuclr.config import NUCLR
from train_nuclr import Trainer

ImportError: cannot import name 'NUCLR' from 'nuclr.config' (/work/submit/kitouni/code/NuCLR/nuclr/config.py)

In [ ]:
import nu

In [ ]:
from nuclr.run import Tracker

Tracker.from_config("config.yaml")